In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# Config for accessing the data on the s3 storage
storage_options = {'anon':True, 'client_kwargs':{'endpoint_url':'https://os.unil.cloud.switch.ch'}}
s3_path = 's3://lts2-graphnex/BXDmice/'

## Genotype
The genotype file contains a list of differences in the genome of the different mice. These differences are at the scale of a nucleotide. In the data table, each row is an `SNP` [Single-nucleotide polymorphism](https://en.wikipedia.org/wiki/Single-nucleotide_polymorphism). It can be inherited from one of the initial ancestors or the other. This is encoded as a binary value, -1 if the `SNP` comes from `C57BL.6J` and 1 from `DBA.2J`. The first descendants of the initial parents, `B6D2F1` and `D2B6F1`, have a zero value since their genome is a perfect mixing with one chromosome from each parent (except for a few SNPs). The stains of the second generation of mice are called `BXD*` (B 'black 6' `C57BL.6J` crossed with D `DBA.2J`).

In [ ]:
# Load the data
# Genotype
genotype_path = os.path.join(s3_path, 'genotype_BXD.txt.gz')
genotype = pd.read_csv(genotype_path, sep='\t', storage_options=storage_options)
print('File {} Opened.'.format(genotype_path))

In [ ]:
genotype.head()

In [ ]:
# Gene postion in the genome
geno_map_path = os.path.join(s3_path, 'map_BXD.txt.gz')
geno_map = pd.read_csv(geno_map_path, sep='\t', storage_options=storage_options)
print('File {} Opened.'.format(geno_map_path))

In [ ]:
geno_map.head()

## Tissues
During or after experiments, the expression of proteins in different tissues of the mice has been measured.
The measurements have been recorded in different files and one file corresponds to one tissue. The dataset inside a file is a large table with proteins as rows and mice as columns. The measurements are float numbers.

For each mouse, only a subset of the tissues have been measured. Therefore, not all mice are present in each tissue measurement dataset and different strains of mice are found in the different tissue files.

In [ ]:
# Tissue
tissue_name = 'LiverProt_CD'
# Other examples:
#tissue_name = 'Eye'
#tissue_name = 'Muscle_CD'
#tissue_name = 'Hippocampus'
#tissue_name = 'Gastrointestinal'
#tissue_name = 'Lung'
tissue_path = os.path.join(s3_path,  'expression data', tissue_name + '.txt.gz')
tissue = pd.read_csv(tissue_path, sep='\t', storage_options=storage_options)
print('File {} Opened.'.format(tissue_path))

In [ ]:
tissue.head()

In [ ]:
# Remove the columns (mouse strains) where there are no measurement:
tissue.dropna(axis=1).head()

## Phenotype
For every BXD strain included in this study, fed with either chow diet (CD) or high fat diet (HFD), measurements describing the phenotype of 3-5 mice were taken. These include body weight and organ mass measurements, activity tests, and molecular measurements (eg. levels of Glucose/ proteins / lipids in blood). The results of these measurements are displayed in a table, with the phenotypes as rows, mouse strains as columns, and values as float numbers. It should be noted that these measurements are displayed in different units (eg. grams for weight, milligrams per liter for blood tests), and value ranges vary. As such, scaling may be advised for further analysis. Moreover, some mouse strains have not been involved in all measurements, resulting in a large number of missing values.
Further information on each phenotype, such as unit of measurement, or associated publication, can be found in a separate table.

In [ ]:
# Load the data
# Phenotype
phenotype_path = os.path.join(s3_path, 'Phenotype.txt.gz')
phenotype = pd.read_csv(phenotype_path, sep='\t', storage_options=storage_options)
print('File {} Opened.'.format(phenotype_path))
# Phenotype description
phenotypeinfo_path = os.path.join(s3_path, 'phenotypes_id_aligner.txt.gz')
phenotypeinfo = pd.read_csv(phenotypeinfo_path, sep='\t', storage_options=storage_options)
print('File {} Opened.'.format(phenotypeinfo_path))

In [ ]:
phenotype.head(10)

In [ ]:
phenotypeinfo.head()

In [ ]:
# Example of one phenotype:
phenotypeinfo[phenotypeinfo['RecordID']==12894]

## Data cleaning

### Drop duplicate genes in the dataset
Some lines in the genotype DataFrame are identical and we will drop them to reduce the number of features and the computation.

In [ ]:
# drop duplicate genes in the dataset
geno_merge = pd.merge(geno_map, genotype, on='SNP')
print('Size of the data before dropping duplicates',geno_merge.shape)
# define a duplicate SNP as: 
# 1) an SNP where all the entries corresponding to BXD mice are identical to another SNP and
# 2) both SNPs are on the same chromosome.
col_to_search_duplicates = ['Chr'] + list(genotype.columns.values[5:])
geno_reduced = geno_merge.drop_duplicates(subset=col_to_search_duplicates)
print('Size of the data after dropping duplicates',geno_reduced.shape)

In [ ]:
# Optionally, save the result as a compressed csv file, to be used by other notebooks
geno_reduced.to_csv('geno_reduced.csv.gz')